In [1]:
import os
import sys
import cv2
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from datetime import datetime
from tqdm import tqdm
from utils import get_params_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.applications import inception_v3, xception, resnet50, vgg16, vgg19
from keras.applications import InceptionV3, Xception, ResNet50, VGG16, VGG19
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [24]:
height = 240
width = 320
train_name = os.listdir('./train')
test_name = os.listdir('./test')
train_size = len(train_name)
test_size = len(test_name)
train = np.zeros((train_size, 240, 320, 3), dtype=np.float64)
test = np.zeros((test_size, 240, 320, 3), dtype=np.float64)
print("Train: %d, Test: %d" % (train_size, test_size))

labels = pd.read_csv('labels.csv')
test_labels = pd.read_csv('sample_submission.csv')
breeds = list(set(labels['breed']))
breeds.sort()
print("Total Dog Breeds:", len(breeds))

Y_train = np.zeros((train_size, len(breeds)), dtype=np.uint8)
for i in range(train_size):
    onehot = breeds.index(labels['breed'][i])
    Y_train[i][onehot] = 1


for i in tqdm(range(train_size)):
    img = cv2.imread('./train/%s.jpg' % labels['id'][i])
    img = cv2.resize(img, dsize=(320, 240))
    img = img[:, :, ::-1]
    train[i] = img
    
for i in tqdm(range(test_size)):
    img = cv2.imread('./test/%s.jpg' % test_labels['id'][i])
    img = cv2.resize(img, dsize=(320, 240))
    img = img[:, :, ::-1]
    test[i] = img
    
print('Training Data Size = %.2f GB' % (sys.getsizeof(train)/1024**3))
print('Testing Data Size = %.2f GB' % (sys.getsizeof(test)/1024**3))

Train: 10222, Test: 10357
Total Dog Breeds: 120


100%|█████████████████████████████████████████████████████| 10357/10357 [00:30<00:00, 338.13it/s]


Training Data Size = 17.55 GB
Testing Data Size = 17.78 GB


In [27]:
train = resnet50.preprocess_input(train)
test = resnet50.preprocess_input(test)

In [25]:
def export_gap(MODEL, preprocess=None, batch_size=128):
    x = Input(shape=(height, width, 3))
    if preprocess is not None:
        x = Lambda(preprocess)(x)
    model = MODEL(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
    train_gap = model.predict(train, batch_size=batch_size)
    test_gap = model.predict(test, batch_size=batch_size)
    with h5py.File("gap_%s.h5" % MODEL.__name__, 'w') as f:
        f.create_dataset('train', data=train_gap)
        f.create_dataset('test', data=test_gap)

In [13]:
export_gap(InceptionV3, inception_v3.preprocess_input)

In [14]:
export_gap(Xception, xception.preprocess_input, 64)

In [16]:
export_gap(InceptionResNetV2, inception_v3.preprocess_input)

In [29]:
export_gap(ResNet50)

## 训练

In [30]:
train_temp = []
test_temp = []
# 'gap_InceptionV3.h5', 'gap_Xception.h5', 'gap_ResNet50.h5', 'gap_InceptionResNetV2.h5'
for gapfile in ['gap_ResNet50.h5']:
    with h5py.File(gapfile, 'r') as f:
        train_temp.append(np.array(f['train']))
        test_temp.append(np.array(f['test']))
train_gap = np.concatenate(train_temp, axis=1)
test_gap = np.concatenate(test_temp, axis=1)
print("Number of Features:", train_gap.shape[1])

Number of Features: 2048


In [31]:
X_train, X_val, y_train, y_val = train_test_split(train_gap, Y_train, shuffle=True, test_size=0.2, random_state=42)

In [32]:
# Input Shape: (Batch Size, Feature Vector length)
x = Input(shape=(X_train.shape[1],))
y = Dropout(0.2)(x)
y = Dense(120, activation='softmax', kernel_initializer='he_normal', name='classifier')(y)
model_gap = Model(inputs=x, outputs=y, name='GAP')
model_gap.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model_gap))

Trainable: 245880, Non-Trainable: 0


In [33]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogBreed-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('gap_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model_gap.fit(x=X_train, y=y_train, batch_size=16, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc, tb])

# model_gap.fit(x=train_gap, y=Y_train, shuffle=True, batch_size=16, epochs=5)

Train on 8177 samples, validate on 2045 samples
Epoch 1/20
8177/8177 [==============================] - 2s 249us/step - loss: 2.5430 - acc: 0.4236 - val_loss: 1.4102 - val_acc: 0.6631
Epoch 2/20
8177/8177 [==============================] - 2s 239us/step - loss: 1.0469 - acc: 0.7454 - val_loss: 0.9897 - val_acc: 0.7413
Epoch 3/20
8177/8177 [==============================] - 2s 215us/step - loss: 0.7279 - acc: 0.8112 - val_loss: 0.8519 - val_acc: 0.7535
Epoch 4/20
8177/8177 [==============================] - 2s 213us/step - loss: 0.5808 - acc: 0.8431 - val_loss: 0.7750 - val_acc: 0.7692
Epoch 5/20
8177/8177 [==============================] - 2s 207us/step - loss: 0.4883 - acc: 0.8663 - val_loss: 0.7373 - val_acc: 0.7736
Epoch 6/20
8177/8177 [==============================] - 2s 201us/step - loss: 0.4289 - acc: 0.8770 - val_loss: 0.7084 - val_acc: 0.7819
Epoch 7/20
8177/8177 [==============================] - 2s 221us/step - loss: 0.3775 - acc: 0.8953 - val_loss: 0.6833 - val_acc: 0.7873


### Fine-tune

In [12]:
X_train, X_val, y_train, y_val = train_test_split(train, Y_train, shuffle=True, test_size=0.2, random_state=42)

In [13]:
def finetune(MODEL, preprocess, height, freeze_till, lr, nb_epoch, opt='SGD', weights=None):
    # Preprocess: Standardization
    x = Input(shape=(height, height, 3))
    x = Lambda(preprocess)(x)

    # Base Model: Freeze all conv layers
    base_model = MODEL(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
    for layer in base_model.layers:
        layer.trainable = True
    if freeze_till is not None:
        for layer in base_model.layers[:freeze_till]:
            layer.trainable = False
    else:
        for layer in base_model.layers:
            layer.trainable = False

    # Customized Classifier
    y = Dropout(0.2)(base_model.output)
    y = Dense(120, activation='softmax', kernel_initializer='he_normal')(y)

    # Full Model: Pre-train Conv + Customized Classifier
    model = Model(inputs=base_model.input, outputs=y, name='Transfer_Learning')
    if opt == 'SGD':
        opt = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    print('Trainable: %d, Non-Trainable: %d' % get_params_count(model))
    
    if weights is not None:
        model.load_weights(weights)
    
    # Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
    log_name = '/DogBreed-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
    log_dir = datetime.now().strftime('transfer_model_%Y%m%d_%H%M')
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)

    es = EarlyStopping(monitor='val_loss', patience=20)
    mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
    tb = TensorBoard(log_dir=log_dir)

    model.fit(x=X_train, y=y_train, batch_size=64, epochs=nb_epoch, 
              validation_data=(X_val, y_val), callbacks=[es, mc, tb])

### SGD, LR 0.045, 1 Epoch

In [14]:
finetune(Xception, xception.preprocess_input, height=299, freeze_till=116, lr=0.045, nb_epoch=1, opt='SGD')

Trainable: 7034264, Non-Trainable: 14073096
Train on 8177 samples, validate on 2045 samples
Epoch 1/1
8177/8177 [==============================] - 87s 11ms/step - loss: 1.3072 - acc: 0.7047 - val_loss: 0.5674 - val_acc: 0.8186


### SGD, LR 0.001, 5 Epochs

In [16]:
log = './transfer_model_20171119_1943/DogBreed-EP01-LOSS0.5674.h5'
finetune(Xception, xception.preprocess_input, height=299, 
         freeze_till=86, lr=0.001, nb_epoch=5, opt='SGD', weights=log)

Trainable: 11876192, Non-Trainable: 9231168
Train on 8177 samples, validate on 2045 samples
Epoch 1/5
8177/8177 [==============================] - 108s 13ms/step - loss: 0.3416 - acc: 0.9016 - val_loss: 0.4369 - val_acc: 0.8670
Epoch 2/5
8177/8177 [==============================] - 106s 13ms/step - loss: 0.2742 - acc: 0.9269 - val_loss: 0.4238 - val_acc: 0.8680
Epoch 3/5
8177/8177 [==============================] - 106s 13ms/step - loss: 0.2518 - acc: 0.9356 - val_loss: 0.4171 - val_acc: 0.8748
Epoch 4/5
8177/8177 [==============================] - 105s 13ms/step - loss: 0.2355 - acc: 0.9424 - val_loss: 0.4102 - val_acc: 0.8778
Epoch 5/5
8177/8177 [==============================] - 105s 13ms/step - loss: 0.2180 - acc: 0.9479 - val_loss: 0.4053 - val_acc: 0.8773


## 导出

In [76]:
model_gap.load_weights('./gap_model_20171015_2231/DogBreed-EP08-LOSS0.2017.h5')

In [85]:
y_test = model_gap.predict(test_gap)

In [86]:
test_labels.iloc[:, 1:] = y_test
test_labels.to_csv('results.csv', index=False)